# First step

making count and TPM matrix using RSEM files.

you need to keep all the RSEM files in folder near to this script called "rsemFiles" and then run this script

In [1]:
import os
import pandas as pd
import collections
import numpy as np
import argparse

In [2]:
# get RSEM files to prepare RNA matrixs(TPM and count)
def prepare_data_RNA_seq(RSEM_directory):
    RSEM_filenames = os.listdir(RSEM_directory)
    geneID = None
    exp = []
    count = []

    for RSEM_file in RSEM_filenames:
        print(RSEM_file)
        metadata_RSEM = RSEM_file.split("_")

        file = pd.read_csv(RSEM_directory + '/' + RSEM_file, sep='\t').values

        if geneID is None:
            geneID = list(file[:, 0])
        elif collections.Counter(geneID) != collections.Counter(file[:, 0]):
            print("order of gene ID is not match!")

        exp.append(list(file[:, 5]))
        count.append(list(file[:, 4]))

    expData = pd.DataFrame(np.array(exp).transpose(), columns=RSEM_filenames, index=geneID)
    countData = pd.DataFrame(np.array(count).transpose(), columns=RSEM_filenames, index=geneID)

    expData.to_csv("expressionList.csv")
    countData.to_csv("countMatrix.csv")

In [3]:
RSEM_directory = "rsemFiles"
prepare_data_RNA_seq(RSEM_directory)

hipp_M_BL6_12mo_7772_S28_rsem.genes.results
hipp_F_BL6_12mo_8123_S32_rsem.genes.results
hipp_F_5xFADHEMI_12mo_7939_S29_rsem.genes.results
hipp_M_PICALM_HO_12mo_8694_S5_rsem.genes.results
hipp_F_5xFADHEMI_PICALM_HO_12mo_9160_S7_rsem.genes.results
hipp_M_5xFADHEMI_PICALM_HO_12mo_8693_S4_rsem.genes.results
hipp_F_PICALM_HO_12mo_8653_S2_rsem.genes.results
hipp_M_5xFADHEMI_PICALM_HO_12mo_8692_S3_rsem.genes.results
hipp_M_BL6_12mo_8350_S33_rsem.genes.results
hipp_M_BL6_12mo_7121_S20_rsem.genes.results
hipp_F_BL6_12mo_7580_S25_rsem.genes.results
hipp_M_PICALM_HO_12mo_9082_S6_rsem.genes.results
hipp_M_5xFADHEMI_12mo_7120_S19_rsem.genes.results
hipp_M_BL6_12mo_8041_S30_rsem.genes.results
hipp_M_PICALM_HO_12mo_9325_S13_rsem.genes.results
hipp_F_PICALM_HO_12mo_9532_S17_rsem.genes.results
hipp_M_5xFADHEMI_12mo_7764_S27_rsem.genes.results
hipp_M_PICALM_HO_12mo_9322_S11_rsem.genes.results
hipp_F_BL6_12mo_8062_S31_rsem.genes.results
hipp_M_PICALM_HO_12mo_8301_S1_rsem.genes.results
hipp_F_5xFADHEMI_12

## clean metadata and sort it 
some times name of the mouse models will be prablamatic due to have some special alphabet 

In [4]:
expressionList = pd.read_csv('expressionList.csv', index_col=0)
countlist = pd.read_csv('countMatrix.csv', index_col=0)

expressionList.columns = expressionList.columns.str.replace('F_PICALMHO', 'F_5xFADWT_PICALMHO')
expressionList.columns = expressionList.columns.str.replace('M_PICALMHO', 'M_5xFADWT_PICALMHO')
expressionList.columns = expressionList.columns.str.replace('5xFADHEMI_PICALM_HO', '5xFADHEMI_PICALMHO')
expressionList.columns = expressionList.columns.str.replace('PICALM_HO', '5xFADWT_PICALMHO')
expressionList.columns = expressionList.columns.str.replace('BL6', '5xFADWT')

col = []
for i in range(expressionList.shape[1]):
    tmp = expressionList.columns[i].split("_")
    if len(tmp) == 7:
        tmp = tmp[3] + "_" + tmp[0] + "_" + tmp[1] + "_" + tmp[2] + "_" + tmp[4] + "_" + tmp[5]
    else:
        tmp = tmp[4] + "_" + tmp[0] + "_" + tmp[1] + "_" + tmp[2] + ":" + tmp[3] + "_" + tmp[5] + "_" + tmp[6]
    col.append(tmp)

countlist.columns = col
expressionList.columns = col

expressionList.to_csv("expressionList.csv")
countlist.to_csv("countMatrix.csv")

## Sort data

In [5]:
### Sort data
expressionList.columns = expressionList.columns.str.replace('4mo', '114mo')
expressionList.sort_index(axis = 1, inplace=True)
expressionList.columns = expressionList.columns.str.replace('114mo', '4mo')

expressionList.to_csv("expressionList_sorted.csv")

## make dataTrait

In [6]:
## create metadata
expressionList = pd.read_csv('expressionList_sorted.csv', index_col=0)
metadata = []

for RSEM_file in expressionList.columns:
    metadata_RSEM = RSEM_file.split("_")
    metadata.append([RSEM_file, metadata_RSEM[0], metadata_RSEM[1], metadata_RSEM[2], metadata_RSEM[3]])

RNA_metadata = pd.DataFrame(metadata, columns=['file name', 'Time point', 'Tissue', 'Sex', 'Group'])
RNA_metadata.to_csv("experimentList_sorted.csv", index=False)